In [ ]:
import weaviate
import weaviate.classes as wvc
import os
import requests

# Set up environment variables
os.environ["WCD_DEMO_URL"] = "YOUR_WEAVIATE_URL"
os.environ["WCD_DEMO_RO_KEY"] =  "YOUR_WEAVIATE_API_KEY"
os.environ["OPENAI_APIKEY"] = "YOUR_OPENAI_API_KEY"

# Cloudflare AI setup
ACCOUNT_ID = ""
os.environ["CLOUDFLARE_AUTH_TOKEN"] = ""
AUTH_TOKEN = os.environ.get("CLOUDFLARE_AUTH_TOKEN")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ["WCD_DEMO_URL"],
    auth_credentials=wvc.init.Auth.api_key(os.environ["WCD_DEMO_RO_KEY"]),
    headers={"X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]}
)

def query_weaviate(topic):
    try:
        questions = client.collections.get("Question")
        response = questions.query.near_text(
            query=topic,
            limit=5
        )
        return [obj.properties for obj in response.objects]
    except Exception as e:
        print(f"Error querying Weaviate: {e}")
        return []

def generate_content(topic, context):
    url = f"https://api.cloudflare.com/client/v4/accounts/{ACCOUNT_ID}/ai/run/@cf/meta/llama-3.1-8b-instruct"
    headers = {
        "Authorization": f"Bearer {AUTH_TOKEN}",
        "Content-Type": "application/json"
    }
    prompt = f"""Generate a Wikipedia-style article about {topic}. Use the following information as context:

{context}

Please structure the article with the following sections:
1. Introduction
2. History
3. Types of {topic}
4. Applications
5. Current State of Research
6. Benefits and Limitations
7. Ethical Considerations
8. Future Prospects

Ensure the content is factual, well-structured, and in an encyclopedic style."""

    data = {"prompt": prompt}
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()['result']['response']
    except requests.exceptions.RequestException as e:
        print(f"Error generating content: {e}")
        return None

def create_wikipedia_article(topic):
    # Query Weaviate for relevant information
    relevant_info = query_weaviate(topic)
    
    # Prepare context from relevant information
    context = "\n".join([f"Q: {info.get('question', '')}\nA: {info.get('answer', '')}" for info in relevant_info])
    
    # If no context was found, use a default message
    if not context:
        context = "No specific context found. Please generate a general article based on common knowledge about the topic."
    
    # Generate content using Cloudflare AI
    article_content = generate_content(topic, context)
    
    return article_content

# Example usage
topic = "Elon Musk"
article = create_wikipedia_article(topic)

if article:
    print(f"Wikipedia-style article about {topic}:\n\n{article}")
else:
    print("Failed to generate article.")

# Close Weaviate client
client.close()
